### gemma-2b-it Instruction Tuning
preprocess에서 전처리한 데이터를 이용해 gemma 모델을 학습시킵니다.
코드들은 [Stanford alpaca](https://github.com/tatsu-lab/stanford_alpaca)를 기반으로 작성되었습니다.
학습 환경은 아래와 같습니다.

|구분|내용|
|-|-|
|학습환경|Google Colab|
|GPU|L4(22.5GB)|
|VRAM|약 17GB 소요|
|dtype|bfloat16|
|Attention|flash attention2|
|Tuning|Lora(r=4, alpha=32)|
|Learning Rate|1e-4|
|Optimizer|adamw_torch_fused|

---

1. Pretrained 모델 설정  
GPU 환경에서는 [flash attention](https://github.com/Dao-AILab/flash-attention)을 사용하는 편이 좋습니다. 속도나 GPU 사용면에서 장점이 있으며, 퍼포먼스의 차이도 없습니다.  
device는 cuda로 설정, dtype은 bfloat16으로 설정하였습니다.

2. 토크나이저를 정의합니다.  
추후 작성할 프롬프트들을 토크나이징하여 input_ids와 labels를 생성합니다.  

3. 마스킹
프롬프트들을 포맷팅한 다음 토크나이징 작업을 수행합니다.  
학습에 사용할 데이터들을 마스킹 작업하여 훈련에 사용할 수 있게 합니다.

4. `Dataset`과 `data_collator`를 정의합니다.  
2번의 토크나이징한 데이터로 `Dataset`을 생성하여 `train_data_set`으로 설정합니다.
`data_collator`를 통해 Sequence의 Padding을 수행합니다.

5. Lora 설정
Peft 학습을 위해 LoraConfig를 설정합니다.

6. 학습
1번의 Pretrained 모델, 4번의 Dataset 및 data_collator, 5번의 LoraConfig를 토대로 학습을 수행합니다.  

7. 기존 모델과 비교  
기존 gemma 모델과 Instruction Tuning된 모델의 답변을 비교해봅니다.

In [3]:
!pip install --quiet\
openai\
selenium\
datasets\
accelerate\
flash-attn\
peft\
trl\
transformers\
python-dotenv\
wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00


In [4]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig
import torch
from datasets import Dataset
from google.colab import userdata, drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/instruction-tuning-with-rag-example"

sys.path.append(path)
import utils
import prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Pretrained 모델 설정
[google/gemma-1.1-2b-it](https://huggingface.co/google/gemma-2b-it)  
구글 Colab Secret 기능으로 허깅페이스 토큰을 사용할 수 있습니다.([Colab Secret 가이드](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75))  
gemma 모델은 토큰이 없으면 받을 수 없습니다.  

학습시 메모리 사용량을 크게 줄일 수 있는 방법은 3가지 정도였습니다.
+ Quantization
+ Flash Attention
+ PEFT

#### Quantization
`torch.bfloat16` 또는 `torch.float16` 타입으로 퍼포먼스를 비교적 떨어뜨리지 않으면서 적은 메모리를 사용할 수 있습니다.  
더 자세한 내용은 [huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/performance#floating-data-types)를 참고하세요.  

#### Flash Attention
GPU를 사용하는 환경이면 flash attention을 사용하길 추천드립니다.  
inference시 15GB정도의 VRAM이 소요되었지만, flash attention을 사용시 5GB 정도로 VRAM 사용량이 감소하였습니다.  
flash attention은 속도와 메모리 사용량을 감소시킬 수 있지만 퍼포먼스의 큰 차이가 없습니다.  
자세한 내용은 [https://arxiv.org/pdf/2307.0869](https://arxiv.org/pdf/2307.0869)을 참조해주세요.

#### PEFT
가장 많이 사용되는 LoRA의 경우 논문에 따르면 GPU 사용량이 1/3 가량으로 절감되었다고 합니다.  
또한 Full Fine Tuning과 비교하여 퍼포먼스의 차이도 크지 않다고 합니다(어떤 경우는 퍼포먼스가 더 좋았다고 합니다).  
자세한 내용은 [https://arxiv.org/pdf/2106.09685](https://arxiv.org/pdf/2106.09685)를 참조해주세요.

In [5]:
model_id = "google/gemma-1.1-2b-it"
token = userdata.get('HF_TOKEN_READ') # Colab Secret에 설정 가능
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=token,
    attn_implementation="flash_attention_2"
)
# 토크나이저의 max_length가 매우 큰수로 지정되어있으므로, 모델의 sequence length인 8192로 세팅
tokenizer.model_max_length = model.config.max_position_embeddings

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 2. tokenizer 설정
프롬프트들을 받아 Embedding index를 반환하는 토크나이저 기능을 정의합니다.

In [6]:
def tokenize(strings, tokenizer):
    tokenized_list = [
        tokenizer(
            text,
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
            padding="longest",
            truncation=True
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

### 3. 마스킹
데이터셋은 아래의 예시와 같이 전달됩니다.  
+ `input_ids`
```python
tensor([  28693, 238264, 236648, 234541,  90621, 235248, 235284, 237029,  47555,
        235265, 171066,  90621, 150483, 236800, 238597,  74715, 239618, 236137,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

+ `labels`
```python
tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

Stanford Alpaca의 예시를 영문으로 변경하면 아래와 같은 형태입니다.  
+ `input_ids`

```plain
</s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

Instruction:
Give three tips for staying healthy.

Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables.
2. Exercise regularly to keep your body active and strong.
3. Get enough sleep and maintain a consistent sleep schedule.</s>
```


+ `labels`

```plain
<MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK>
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.</s>
```
`<MASK>`는 임의로 적어둔 것이며, 인코딩 토큰 값처럼 -100 값으로 채워지게 됩니다.  
-100은 Pytorch의 Loss 함수에서 `ignore_index`로 사용되는 값이므로, Loss 계산시 제외됩니다.  
따라서 `input_ids`는 완전한 Sequence로 이루어지며, `labels`는 Resonpose 앞쪽이 마스킹된 형태가 됩니다.

In [7]:
# 파이토치에서 Loss 함수들의 ignore_index 값들은 모두 -100으로 처리되어 있습니다
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
IGNORE_INDEX = -100

def preprocess(
    sources,
    examples,
    tokenizer,
):
    """ 프롬프트 데이터를 받아서 전처리"""
    examples_tokenized, sources_tokenized = [tokenize(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

### 4. `dataset`과 `collator` 정의
+ 데이터셋을 로드하여 프롬프트에 포맷팅합니다.
+ 포맷팅한 시퀀스를 `tokenizer`로 인코딩 및 마스킹 처리합니다.
+ 인코딩 및 마스킹 처리가 된 데이터를 `Dataset` 객체로 정의합니다.
+ batch 데이터를 처리하기 위해 `data_collator`를 정의합니다.
+ 데이터셋과 `data_collator`를 `trainer`에 전달하기 쉽게 `dict` 객체로 처리합니다.

In [8]:
from torch.utils.data import Dataset

class SupervisedDataset(Dataset):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""
    def __init__(self, tokenizer, data_path):
        super().__init__()
        instructions = utils.jload(data_path)
        sources = []
        examples = []
        for idx in range(len(instructions)):
            question = json.loads(instructions[idx])['question']
            answer = json.loads(instructions[idx])['answer']
            source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
            example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
            sources.append(source)
            examples.append(example)

        data_dict = preprocess(sources, examples, tokenizer)
        self.input_ids = data_dict['input_ids']
        self.labels = data_dict['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


`Dataset`은 huggingface의 Dataset 클래스를 사용하는 방법도 있습니다.  
아래 코드 역시 `train_dataset`을 생성하여 datasets.Dataset 객체로 생성합니다.  
`Dataset.from_dict` 메서드를 사용하면 별도의 클래스 설정 없이 간단하게 `Dataset`을 생성합니다.  

```python
def make_train_dataset(data_path: str):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""

    instructions = utils.jload(data_path)
    sources = []
    examples = []
    for idx in range(len(instructions)):
        question = json.loads(instructions[idx])['question']
        answer = json.loads(instructions[idx])['answer']
        source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
        example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
        sources.append(source)
        examples.append(example)

    data_dict = preprocess(sources, examples, tokenizer)
    train_dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format("torch")

    return train_dataset
```

In [9]:
class DataCollatorForSupervisedDataset(object):
    """데이터 콜레이터"""

    def __init__(self, tokenizer):
        self.tokenizer=tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids_ = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels_ = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids_,
            labels=labels_,
            attention_mask=input_ids_.ne(self.tokenizer.pad_token_id),
        )

In [10]:
def make_supervised_data_module(data_path: str, tokenizer):
    """학습에 사용할 데이터셋과 콜레이터를 정의합니다."""
    train_dataset = SupervisedDataset(data_path=data_path, tokenizer=tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

### 5. LoRA PEFT
메모리 절감을 최대화 하기 위해 모델 내 Adapter를 추가할 수 있는 모든 Layer에 LoRA를 적용하였습니다.
+ Layer이름은 모델마다 다르니 모델의 Layer를 확인하고 추가해주세요.
+ `TrainingArguments`의 경우, 여러번의 학습을 통해 적절한 하이퍼파라미터를 찾아서 학습시켰습니다.  

아래 코드는 [huggingface의 블로그](https://huggingface.co/blog/gemma-pefthttps://huggingface.co/blog/gemma-peft)를 참조하였습니다.

파라미터에 대한 설명은 huggingface에서 확인해주세요.  
+ [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
+ [LoraConfig](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)
+ [SFTTRainer](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)  

파라미터를 수정하면서 어떻게 모델이 훈련되는지 지켜보는 것도 좋은 방법입니다.  
또한 파라미터를 수정해보면서 GPU에 어떤 영향을 주는지 보는 것도 좋습니다.  


In [16]:
from transformers import TrainingArguments

my_model_name = os.path.join(path, "gemma-2b-it-example-v1")

args = TrainingArguments(
    output_dir=my_model_name,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    bf16=True,
    push_to_hub=True,
    hub_token=userdata.get("HF_TOKEN"),
    report_to="wandb",
    run_name="gemma-2b-it-example-v1"
)

In [17]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.0,
        r=4,
        bias="none",
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

### 6. 학습

wandb 사용법은 [wandb guide](https://docs.wandb.ai/guides/integrations/huggingface) 페이지를 참조해주세요.

In [18]:
import wandb

wandb.login()

True

In [19]:
from trl import SFTTrainer

data_path = os.path.join(path, "data/instruction.jsonl")
max_seq_length = model.config.max_position_embeddings
data_module = make_supervised_data_module(data_path=data_path, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    **data_module,
)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.025000
20,2.087500
30,2.100000
40,1.875000
50,1.725000
60,1.625000
70,1.500000
80,1.525000
90,1.418700
100,1.456300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

In [5]:
# google/gemma-1.1-2b-it
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
# 학습한 파인튜닝 모델
finetuned_model = AutoModelForCausalLM.from_pretrained(
    os.path.join(path, "gemma-2b-it-example-v1"),
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
from utils import generate

rag_config = {
    "api_client_id": userdata.get('NAVER_API_ID'),
    "api_client_secret": userdata.get('NAVER_API_SECRET')
    }

🚀 기존 Gemma Instruction 모델

In [9]:
query = "전세사기에 대처하는 방법"

In [10]:
# 훈련 전 모델
completion = generate(
    model=model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
print(completion)

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


user
전세사기에 대처하는 방법
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
-현직 공인중개사가 알려주는 부동산 전세사기 대처방법을 알려드립니다. 전세사기란,,,,??? 전입신고, 확정일자를 받아도 이 기존 법의 허점을 역이용하여 다시 대출을 받거나, 보증금을 그냥 꿀꺽하는 전개로... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (

🚀 Instruction Tuning된 모델

In [13]:
# 훈련 전 모델
completion = generate(
    model=finetuned_model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
print(completion)

user
전세사기에 대처하는 방법
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
-현직 공인중개사가 알려주는 부동산 전세사기 대처방법을 알려드립니다. 전세사기란,,,,??? 전입신고, 확정일자를 받아도 이 기존 법의 허점을 역이용하여 다시 대출을 받거나, 보증금을 그냥 꿀꺽하는 전개로... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (

In [14]:
# model.generate
input_text = "아파트 재건축에 대해 알려줘."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = finetuned_model.generate(**input_ids, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))


<bos>아파트 재건축에 대해 알려줘.

A. 재건축 조합 설립과 재건축 조합원 지위권 설정
B. 재건축 조합원 지위권 설정과 재건축 조합 설립
C. 재건축 조합원 지위권 설정과 재건축 조합 설립
D. 재건축 조합원 지위권 설정과 재건축 조합 설립

Answer: C

Explanation: 재건축 조합원 지위권 설정과 재건축 조합 설립은 재건축 조합원 지위권 설정과 재건축 조합 설립이 동시에 이루어집니다. 재건축 조합원 지위권 설정은 재건축 조합원이 조합원 지위권을 승계받는 절차로, 재건축 조합 설립은 조합원들이 재건축 사업을 통해 얻는 자산을 조합원 지위권으로 승계받는 절차입니다. 따라서, 재건축 조합원 지위권 설정과 재건축 조합 설립은 동시에 이루어지며, 재건축 조합원 지위권 설정과 재건축 조합 설립이 서로 연관이 있습니다.<eos>


In [16]:
with open(os.path.join(path, "data/eval_dataset.txt"), "r") as f:
    eval_inputs = f.readlines()

In [17]:
eval_inputs = [inputs.strip("\n") for inputs in eval_inputs]

In [28]:
from peft import AutoPeftModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

finetuned_model = AutoModelForCausalLM.from_pretrained(
    "aiqwe/gemma-2b-it-example-v1",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
eval_inputs

['정비 사업의 진행 절차를 알려줘.',
 '실거주 의무가 뭔가요?',
 '신속통합기획 진행중인 곳이 어디인가요?',
 '청약에 당첨되려면 뭘 해야할까요?',
 '부동산 주요 정책은 뭐가 있나요?',
 '전세 사기에 대비하는 방법을 알려주세요.',
 '부동산 매도시 양도세에 대해 설명해주세요.',
 '전매 제한이 뭔가요?',
 '주택 담보 대출을 받으려면 어떻게 해야할까요?',
 'DSR, DTI, LTV에 대해 설명해주세요.',
 '디에이치 방배는 언제 분양되나요?',
 '한남 뉴타운 진행사항이 어떻게 될까요?',
 '분양가 상한제시 어떤 규제를 받나요?',
 '임차인 우서 변제권이 뭔가요?',
 '전세 중개수수료는 얼마나 내야하나요?']

In [29]:
# model.generate
inputs = eval_inputs
inputs_with_tag = [instruction + "<ANSWER>" for instruction in inputs]

input_text = tokenizer(inputs_with_tag, return_tensors="pt", padding=True).to(base_model.device)

outputs = base_model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5)
base_decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
base_decoded = [decoded.split("<ANSWER>")[1] for decoded in base_decoded]

outputs = finetuned_model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5)
finetuned_decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
finetuned_decoded = [decoded.split("<ANSWER>")[1] for decoded in finetuned_decoded]

eval_dataset = dict(input1=inputs, completion1=base_decoded, input2=inputs, completion2=finetuned_decoded)
utils.jsave(data=eval_dataset, save_path=os.path.join(path, "data/eval_dataset.json"), mode="a", indent=4)

In [30]:
result = utils.jload(os.path.join(path, "data/eval_dataset.json"))

In [35]:
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
client = OpenAI()

def get_completion(query):
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="gpt-4o"
    )
    return completion.choices[0].message.content

In [36]:
PROMPT = """
[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below.
You should choose the assistant that follows the user’s instructions and answers the user’s question better.
You should judge whether the information is fake or not based on your biggest consideration.
Be as objective as possible.
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision.
Do not allow the length of the responses to influence your evaluation. Do not favor certain names of assistants.
Be as objective as possible.
After evaluation, output of your final verdict by strictly following this format:
"A" if assistant A is better, "B" if assistant B is better, and "C" for a tie.
Just write the preference only like "A", "B" or "C" without any explain.

[User Question]
{question}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]
"""

In [39]:
eval_dataset = utils.jload("/content/drive/MyDrive/instruction-tuning-with-rag-example/data/eval_dataset.json")

In [57]:
scores = []
for idx in range(len(eval_dataset['input1'])):
    q = eval_dataset['input1'][idx]
    c1 = eval_dataset['completion1'][idx]
    c2 = eval_dataset['completion2'][idx]
    res = get_completion(PROMPT.format(question=q, answer_a=c1, answer_b=c2))
    scores.append(res)


In [58]:
scores

['A', 'B', 'B', 'B', 'C', 'B', 'C', 'B', 'C', '"B"', 'C', 'B', 'B', 'C', '"C"']

In [59]:
scores = []
for idx in range(len(eval_dataset['input1'])):
    q = eval_dataset['input1'][idx]
    c1 = eval_dataset['completion1'][idx]
    c2 = eval_dataset['completion2'][idx]
    res = get_completion(PROMPT.format(question=q, answer_a=c2, answer_b=c1))
    scores.append(res)

In [60]:
scores

['A', '“A”', 'A', '"A"', 'C', 'A', 'A', 'A', 'C', 'B', 'C', 'B', 'C', 'C', 'C']

In [ ]:
scores = []
for evals in eval_dataset:
    res = get_completion(PROMPT.format(question=data['input1'], answer_a=data['completion1'], answer_b=data['completion2']))
    scores.append(res)

In [ ]:
scores

['B', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A']

In [ ]:
scores

['A', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'B']

In [ ]:
scores

['B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B']

In [ ]:
scores

['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']

In [ ]:
scores

['A', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B']

In [ ]:
scores

['B', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B']

In [ ]:
scores

['A', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'A', 'B']

In [ ]:
scores

['B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B']

In [ ]:
scores

['B', 'A', 'B', 'A', 'B', 'A', 'A', 'A', 'A', 'B']

In [ ]:
scores

['B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'B']

In [ ]:
idx = 4
print(json.loads(eval_dataset[idx])['input1'])
print("Answer1")
print(json.loads(eval_dataset[idx])['completion1'])
print("Answer2")
print(json.loads(eval_dataset[idx])['completion2'])

마포 자이 힐스테이트 언제 분양되나요?
Answer1


마포 자이 힐스테이트는 현재까지 공식적으로 발급되지 않고 있습니다.
Answer2

마포 자이 힐스테이트의 입양 및 분양 시설은 2018년에 완성되었으며, 분양 일정은 아래와 같습니다.

* 2023년 4월: 예비 입양자에게 공고된 사례입니다.


In [ ]:
scores[9]

"Assistant A's response provides a detailed and comprehensive plan on how to prevent falling victim to fraud in the real estate market. It covers various aspects such as education, research, technological understanding, product development, market analysis, branding, and policy support, which are all crucial in safeguarding oneself against deception.\n\nAssistant B's response mainly focuses on understanding the risks of fraud, learning from real cases, receiving education on fraud prevention, being cautious with payments, and making careful investment decisions. While these points are relevant, they lack the depth and breadth of information provided by Assistant A.\n\nOverall, Assistant A offers a more thorough and structured approach to prevent fraud in the real estate market, making it the better choice in addressing the user's question.\n\nTherefore, my verdict is:\nA"

In [49]:
SCORE_PROMPT="""
[System]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response.
Most of all, you should consider hallucination of the answer. Be as objective as possible.
After evaludation, rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".
Just write the score only.
[Question]
{question}
[The Start of Assistant’s Answer]
{answer}
[The End of Assistant’s Answer]
"""

In [53]:
scores = []
for idx in range(len(eval_dataset['input1'])):
    q = eval_dataset['input1'][idx]
    c1 = eval_dataset['completion1'][idx]
    c2 = eval_dataset['completion2'][idx]
    res = get_completion(SCORE_PROMPT.format(question=q, answer=c1))
    scores.append(res)


In [55]:
scores = []
for idx in range(len(eval_dataset['input1'])):
    q = eval_dataset['input1'][idx]
    c1 = eval_dataset['completion1'][idx]
    c2 = eval_dataset['completion2'][idx]
    res = get_completion(SCORE_PROMPT.format(question=q, answer=c2))
    scores.append(res)

In [54]:
scores

['[[3]]',
 'Rating: [[1]]',
 '[[2]]',
 'Rating: [[1]]',
 'Rating: [[3]]',
 '[[1]]',
 '[[2]]',
 '[[3]]',
 'Rating: [[4]]',
 '[[1]]',
 '[[1]]',
 'Rating: [[2]]',
 'Rating: [[1]]',
 'Rating: [[2]]',
 'Rating: [[2]]']

In [56]:
scores

['Rating: [[4]]',
 '[[2]]',
 '[[4]]',
 'Rating: [[4]]',
 'Rating: [[2]]',
 'Rating: [[3]]',
 'Rating: [[2]]',
 'Rating: [[5]]',
 'Rating: [[5]]',
 '[[1]]',
 '[[1]]',
 'Rating: [[3]]',
 '[[2]]',
 '[[2]]',
 'Rating: [[2]]']

In [ ]:
3 2 3 2 2 2 3 2 1 2